# 📝 Ajout de Cas Manuels (basés sur PDF)

**Objectif :** Augmenter le dataset en ajoutant des cas manuels .

**Avantage :** Rapide, contrôlé, pas de rate limit !

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pickle
from pathlib import Path



c:\Users\HP\Desktop\Master2\llm\Triage_urgence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1️⃣ Charger CamemBERT

In [2]:
model_name = "almanach/camembert-bio-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()



Loading weights: 100%|██████████| 197/197 [00:00<00:00, 630.87it/s, Materializing param=encoder.layer.11.output.dense.weight]              
CamembertModel LOAD REPORT from: almanach/camembert-bio-base
Key                             | Status     | 
--------------------------------+------------+-
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
pooler.dense.weight             | MISSING    | 
pooler.dense.bias               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


CamembertModel(
  (embeddings): CamembertEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
  )
  (encoder): CamembertEncoder(
    (layer): ModuleList(
      (0-11): 12 x CamembertLayer(
        (attention): CamembertAttention(
          (self): CamembertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): CamembertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
     

## 2️⃣ Fonction pour créer cas manuel

In [3]:
def create_manual_case(symptoms, age, sexe, fc, fr, spo2, ta_sys, ta_dia, temp, label):
    """Crée un cas manuel avec embeddings."""
    
    # Encoder symptômes
    text = ", ".join(symptoms)
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    emb = outputs.last_hidden_state[:, 0, :].numpy()[0]
    
    # Features
    features = list(emb)
    features.extend([
        (fc - 70) / 30,
        (fr - 16) / 5,
        (spo2 - 95) / 5,
        (ta_sys - 120) / 20,
        (ta_dia - 80) / 10,
        (temp - 37) / 2,
        (age - 50) / 25,
        1 if sexe == 'M' else 0
    ])
    
    return np.array(features), label

print("✅ Fonction créée")

✅ Fonction créée


## 3️⃣ Cas manuels (basés sur PDF)

In [4]:
#  creer des cas manuels 

MANUAL_CASES = [
    
    # ════════════════════════════════════════════════════════════
    #  ROUGE - Potentiellement vital et urgent (15 cas)
    # ════════════════════════════════════════════════════════════
    
    # 1. Douleur thoracique
    (["douleur thoracique intense", "oppressante"], 65, "M", 120, 24, 88, 90, 60, 37.0, "ROUGE"),
    
    # 2. Difficulté respiratoire sévère
    (["difficulté respiratoire sévère", "essoufflement au repos"], 58, "F", 130, 32, 85, 140, 90, 37.2, "ROUGE"),
    
    # 3. Perte de conscience
    (["perte de conscience", "évanouissement"], 45, "M", 110, 20, 92, 100, 65, 37.5, "ROUGE"),
    
    # 4. Paralysie soudaine
    (["paralysie soudaine côté droit", "visage affaissé"], 72, "M", 95, 18, 94, 160, 95, 37.0, "ROUGE"),
    
    # 5. Difficulté à parler
    (["difficulté soudaine à parler", "confusion importante"], 68, "F", 100, 19, 93, 150, 90, 36.8, "ROUGE"),
    
    # 6. Convulsions
    (["convulsions", "crise d'épilepsie"], 35, "M", 125, 22, 91, 130, 85, 37.8, "ROUGE"),
    
    # 7. Hémorragie importante
    (["hémorragie importante", "sang qui coule abondamment"], 42, "M", 135, 26, 87, 80, 50, 36.5, "ROUGE"),
    
    # 8. Traumatisme crânien avec perte de conscience
    (["traumatisme crânien", "perte de conscience"], 28, "M", 115, 20, 90, 110, 70, 37.0, "ROUGE"),
    
    # 9. Douleur abdominale très intense
    (["douleur abdominale très intense", "ventre dur"], 55, "F", 120, 24, 89, 95, 60, 37.5, "ROUGE"),
    
    # 10. Réaction allergique grave
    (["gonflement du visage", "difficulté à respirer", "allergie"], 30, "F", 130, 30, 86, 100, 65, 37.0, "ROUGE"),
    
    # 11. Perte de vision soudaine
    (["perte de vision soudaine"], 62, "M", 90, 18, 95, 145, 90, 37.0, "ROUGE"),
    
    # 12. Vertiges intenses
    (["vertiges intenses", "impossibilité de tenir debout"], 58, "F", 105, 20, 93, 85, 55, 36.8, "ROUGE"),
    
    # 13. Vomissements de sang
    (["vomissements de sang"], 48, "M", 125, 24, 88, 90, 60, 37.2, "ROUGE"),
    
    # 14. Brûlure étendue 3ème degré
    (["brûlure étendue", "3ème degré"], 35, "M", 130, 28, 87, 110, 70, 38.5, "ROUGE"),
    
    # 15. Intoxication
    (["intoxication médicamenteuse", "confusion", "nausées"], 22, "F", 115, 22, 90, 100, 65, 37.0, "ROUGE"),
    
    
    # ════════════════════════════════════════════════════════════
    #  JAUNE - Non vital mais urgent (14 cas)
    # ════════════════════════════════════════════════════════════
    
    # 1. Fracture suspectée bras
    (["fracture suspectée", "douleur intense au bras"], 35, "M", 90, 18, 98, 120, 80, 36.8, "JAUNE"),
    
    # 2. Luxation épaule
    (["luxation de l'épaule", "douleur intense"], 28, "M", 95, 18, 99, 125, 82, 37.0, "JAUNE"),
    
    # 3. Coupure profonde
    (["coupure profonde", "nécessite points de suture"], 42, "F", 88, 17, 98, 118, 78, 36.9, "JAUNE"),
    
    # 4. Brûlure 2ème degré avec cloques
    (["brûlure 2ème degré", "cloques étendues"], 32, "F", 92, 19, 97, 122, 80, 37.5, "JAUNE"),
    
    # 5. Fièvre élevée persistante
    (["fièvre élevée persistante"], 28, "M", 100, 20, 96, 115, 75, 39.5, "JAUNE"),
    
    # 6. Douleur abdominale intense
    (["douleur abdominale intense", "persistante"], 45, "F", 95, 19, 97, 125, 82, 37.8, "JAUNE"),
    
    # 7. Vomissements répétés avec déshydratation
    (["vomissements répétés", "signes de déshydratation"], 22, "F", 105, 20, 96, 100, 65, 37.5, "JAUNE"),
    
    # 8. Crise d'asthme modérée
    (["crise d'asthme modérée", "essoufflement"], 35, "M", 100, 24, 93, 120, 80, 37.0, "JAUNE"),
    
    # 9. Saignement qui ne s'arrête pas
    (["saignement qui ne s'arrête pas"], 38, "F", 95, 18, 96, 110, 72, 37.0, "JAUNE"),
    
    # 10. Traumatisme crânien avec confusion
    (["traumatisme crânien", "confusion légère", "conscient"], 25, "M", 90, 18, 97, 125, 82, 37.2, "JAUNE"),
    
    # 11. Infection urinaire avec fièvre
    (["infection urinaire", "fièvre", "douleurs lombaires"], 50, "F", 92, 18, 98, 128, 84, 38.8, "JAUNE"),
    
    # 12. Entorse grave
    (["entorse grave", "impossibilité de poser le pied"], 30, "M", 85, 17, 99, 120, 80, 36.9, "JAUNE"),
    
    # 13. Réaction allergique avec gonflement
    (["réaction allergique", "gonflement important", "respiration normale"], 28, "F", 88, 17, 98, 118, 78, 37.0, "JAUNE"),
    
    # 14. Coliques néphrétiques
    (["coliques néphrétiques", "douleur intense aux reins"], 42, "M", 95, 19, 97, 135, 88, 37.5, "JAUNE"),
    
    
    # ════════════════════════════════════════════════════════════
    #  VERT - Non vital et non urgent (15 cas)
    # ════════════════════════════════════════════════════════════
    
    # 1. Entorse cheville légère
    (["entorse de cheville légère à modérée"], 28, "M", 75, 16, 99, 120, 80, 36.8, "VERT"),
    
    # 2. Conjonctivite
    (["conjonctivite", "œil rouge et irrité"], 35, "F", 72, 15, 99, 118, 78, 36.9, "VERT"),
    
    # 3. Infection urinaire simple
    (["infection urinaire simple", "sans fièvre"], 45, "F", 78, 16, 98, 122, 82, 37.2, "VERT"),
    
    # 4. Douleur dentaire
    (["douleur dentaire supportable"], 32, "M", 76, 16, 99, 120, 80, 36.7, "VERT"),
    
    # 5. Petite brûlure 1er degré
    (["petite brûlure du 1er degré", "type coup de soleil"], 25, "F", 74, 16, 99, 115, 75, 37.5, "VERT"),
    
    # 6. Mal de dos
    (["mal de dos depuis quelques jours", "stable"], 42, "M", 76, 16, 99, 125, 82, 36.8, "VERT"),
    
    # 7. Gastro-entérite
    (["gastro-entérite", "nausées et diarrhée légère"], 30, "F", 80, 17, 98, 118, 78, 37.8, "VERT"),
    
    # 8. Otite
    (["otite sans fièvre élevée"], 8, "M", 82, 18, 99, 110, 70, 37.5, "VERT"),
    
    # 9. Éruption cutanée
    (["éruption cutanée qui démange"], 28, "F", 74, 16, 99, 118, 78, 36.9, "VERT"),
    
    # 10. Saignement de nez
    (["saignement de nez qui s'arrête facilement"], 22, "M", 75, 16, 99, 120, 80, 36.8, "VERT"),
    
    # 11. Corps étranger dans l'œil
    (["petit corps étranger dans l'œil", "poussière"], 35, "M", 76, 16, 99, 122, 82, 36.9, "VERT"),
    
    # 12. Urticaire localisée
    (["réaction allergique légère", "urticaire localisée"], 32, "F", 78, 16, 99, 118, 78, 37.0, "VERT"),
    
    # 13. Vomissements occasionnels
    (["vomissements occasionnels", "sans déshydratation"], 25, "F", 80, 17, 98, 115, 75, 37.2, "VERT"),
    
    # 14. Douleur abdominale légère
    (["douleur abdominale légère et stable"], 38, "M", 76, 16, 99, 120, 80, 36.8, "VERT"),
    
    # 15. Migraine
    (["migraine habituelle"], 42, "F", 74, 16, 99, 125, 82, 36.9, "VERT"),
    
    
    # ════════════════════════════════════════════════════════════
    #  GRIS - Ne nécessite pas les urgences (15 cas)
    # ════════════════════════════════════════════════════════════
    
    # 1. Renouvellement ordonnance
    (["renouvellement d'ordonnance pour médicaments habituels"], 55, "F", 72, 15, 99, 120, 80, 36.7, "GRIS"),
    
    # 2. Certificat médical sport
    (["demande de certificat médical pour le sport"], 22, "M", 70, 15, 99, 118, 78, 36.8, "GRIS"),
    
    # 3. Petite coupure superficielle
    (["petite coupure superficielle qui a arrêté de saigner"], 28, "F", 74, 16, 99, 115, 75, 36.7, "GRIS"),
    
    # 4. Rhume
    (["rhume ou grippe sans fièvre élevée"], 32, "M", 75, 16, 99, 120, 80, 37.0, "GRIS"),
    
    # 5. Égratignure
    (["petite égratignure ou écorchure"], 18, "F", 72, 15, 99, 118, 78, 36.8, "GRIS"),
    
    # 6. Vaccination routine
    (["demande de vaccination de routine"], 35, "M", 74, 16, 99, 120, 80, 36.9, "GRIS"),
    
    # 7. Mal de gorge léger
    (["mal de gorge léger depuis 1-2 jours"], 28, "F", 75, 16, 99, 118, 78, 37.0, "GRIS"),
    
    # 8. Verrue
    (["petite verrue à retirer"], 25, "M", 72, 15, 99, 120, 80, 36.7, "GRIS"),
    
    # 9. Résultats d'analyses
    (["consultation pour résultats d'analyses déjà faites"], 50, "F", 74, 16, 99, 122, 82, 36.8, "GRIS"),
    
    # 10. Toux légère
    (["toux légère sans difficulté respiratoire"], 32, "M", 76, 16, 99, 118, 78, 36.9, "GRIS"),
    
    # 11. Problème de peau mineur
    (["conseil pour un problème de peau mineur", "bouton", "rougeur légère"], 28, "F", 74, 16, 99, 115, 75, 36.8, "GRIS"),
    
    # 12. Arrêt de travail
    (["demande d'arrêt de travail pour maladie bénigne"], 38, "M", 75, 16, 99, 120, 80, 36.7, "GRIS"),
    
    # 13. Piqûre d'insecte
    (["petite piqûre d'insecte sans réaction importante"], 22, "F", 72, 15, 99, 118, 78, 36.8, "GRIS"),
    
    # 14. Mal de tête occasionnel
    (["mal de tête occasionnel et léger"], 35, "M", 74, 16, 99, 120, 80, 36.9, "GRIS"),
    
    # 15. Fatigue générale
    (["fatigue générale sans autres symptômes"], 42, "F", 76, 16, 99, 122, 82, 36.8, "GRIS"),
]

print(f" {len(MANUAL_CASES)} cas manuels définis")

 59 cas manuels définis


## 4️⃣ Générer features

In [5]:
from tqdm import tqdm

X_manual = []
y_manual = []

print(" Génération features...")

for case in tqdm(MANUAL_CASES):
    features, label = create_manual_case(*case)
    X_manual.append(features)
    y_manual.append(label)

X_manual = np.array(X_manual)
y_manual = np.array(y_manual)

print(f"\n Features générées !")
print(f"   X shape : {X_manual.shape}")
print(f"   y shape : {y_manual.shape}")

 Génération features...


100%|██████████| 59/59 [00:03<00:00, 15.43it/s]


 Features générées !
   X shape : (59, 776)
   y shape : (59,)


## 5️⃣ Distribution

In [6]:
from collections import Counter

counts = Counter(y_manual)

print(" DISTRIBUTION")
print("="*60)
for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']:
    count = counts[label]
    pct = (count / len(y_manual)) * 100
    print(f"   {label:6s} : {count:2d} ({pct:5.1f}%)")
print("="*60)

 DISTRIBUTION
   ROUGE  : 15 ( 25.4%)
   JAUNE  : 14 ( 23.7%)
   VERT   : 15 ( 25.4%)
   GRIS   : 15 ( 25.4%)


## 6️⃣ Combiner avec dataset existant

In [7]:
# Charger dataset généré si existe
data_path = Path('../data/triage_dataset_balanced.pkl')

if data_path.exists():
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    
    X_generated = data['X']
    y_generated = data['y']
    
    # Combiner
    X_combined = np.vstack([X_generated, X_manual])
    y_combined = np.concatenate([y_generated, y_manual])
    
    print(f"✅ Dataset combiné !")
    print(f"   Générés : {len(X_generated)} cas")
    print(f"   Manuels : {len(X_manual)} cas")
    print(f"   TOTAL : {len(X_combined)} cas")
else:
    X_combined = X_manual
    y_combined = y_manual
    print(f"✅ Dataset manuel seul : {len(X_combined)} cas")

✅ Dataset combiné !
   Générés : 20 cas
   Manuels : 59 cas
   TOTAL : 79 cas


## 7️⃣ Sauvegarder

In [8]:
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

with open(data_dir / 'triage_dataset_complete.pkl', 'wb') as f:
    pickle.dump({'X': X_combined, 'y': y_combined}, f)

print(f" Dataset complet sauvegardé !")
print(f"   {data_dir / 'triage_dataset_complete.pkl'}")
print(f"   {len(X_combined)} cas au total")

 Dataset complet sauvegardé !
   ..\data\triage_dataset_complete.pkl
   79 cas au total
